![ibm cloud logo](../images/ibm-cloud.png)

# CASE STUDY - unsupervised learning


In [1]:
import os
import joblib
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import BayesianGaussianMixture
from sklearn.svm import SVC
import imblearn.pipeline as pl
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, SVMSMOTE
    
    
plt.style.use('seaborn')
%matplotlib inline

## Synopsis

  > We are now going to predict customer retention.  There are many models and many transforms to consider.  Use your
    knowledge of pipelines and functions to ensure that your code makes it easy to compare and iterate.  
    
  > Marketing has asked you to make a report on customer retention.  They would like you to come up with information     that can be used to improve current marketing strategy efforts.  The current plan is for marketing at AAVAIL to
    collect more features on subscribers the and they would like to use your report as a proof-of-concept in order to     get buyin for this effort.
  
## Outline

1. Create a churn prediction baseline model
2. Use clustering as part of your prediction pipeline
3. 
4. Run and experiment to see if re-sampling techniques improve your model

## Data

Here we load the data as we have already done.

`aavail-target.csv`

In [2]:
df = pd.read_csv(os.path.join(".",r"aavail-target.csv"))

## pull out the target and remove uneeded columns
_y = df.pop('is_subscriber')
y = np.zeros(_y.size)
y[_y==0] = 1 
df.drop(columns=['customer_id','customer_name'],inplace=True)
df.head()

,country,age,subscriber_type,num_streams
0,united_states,21,aavail_premium,23
1,singapore,30,aavail_unlimited,12
2,united_states,21,aavail_premium,22
3,united_states,20,aavail_basic,19
4,singapore,21,aavail_premium,23


### QUESTION 1

Create a stratified train test split of the data

In [6]:
## YOUR CODE HERE
rs = 1
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25, stratify=y, random_state=rs)

### QUESTION 2

Create a baseline model.  We are going to test whether clustering
followed by a model improves the results.  The we will test whether
re-sampling techniques provide improvements.  Use a pipeline or
another method, but create a baseline model given the data. Here is
the ColumnTransformer we have used before.

In [4]:
## preprocessing pipeline
numeric_features = ['age', 'num_streams']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_features = ['country', 'subscriber_type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [11]:
## YOUR CODE HERE
param_grid_svm = {
    'svm__C': [0.01,0.1,0.5,1.0,1.5,5.0,10.0],
    'svm__gamma': [0.001,0.01,0.1]
}
best_params = {}
pipe_svm = Pipeline(steps=[('pre', preprocessor),
                           ('svm',SVC(kernel='rbf',
                                      class_weight='balanced'))])
grid = GridSearchCV(pipe_svm,
                    param_grid=param_grid_svm,
                    cv=5)
grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)

print("-->".join(pipe_svm.named_steps.keys()))
best_params = grid.best_params_

print("f1_score",round(f1_score(y_test, y_pred,average='binary'),3))

param_grid_rf = {
    'rf__n_estimators': [20,50,100,150],
    'rf__max_depth': [4, 5, 6, 7, 8],
    'rf__criterion': ['gini', 'entropy']
}

pipe_rf = Pipeline(steps=[('pre', preprocessor),
                          ('rf',RandomForestClassifier())])

grid = GridSearchCV(pipe_rf,
                    param_grid=param_grid_rf,
                    cv=5)

grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print("-->".join(pipe_rf.named_steps.keys()))
best_params = dict(best_params, **grid.best_params_)
print("f1_score",round(f1_score(y_test, y_pred,average='binary'),3))

pre-->svm
f1_score 0.609


pre-->rf
f1_score 0.538


### QUESTION 3

The next part is to create version of the classifier that uses
identified clusters. I.e. you would create 4 new features based on 4
identified clusters in the data and you would use such clustering as
new feature for the classification job as performed above. This is
similar to the idea you had in the project when you wanted to classify
the input of fire, health etc. and then based on that perform your
classificiation task.

Here is a class to get you started.  It is a transformer like those
that we have been working with.

There is an example of how to use it just below.  In this example 4
clusters were specified and their one-hot encoded versions were
appended to the feature matrix.  Now using pipelines and/or functions
compare the performance using cluster profiling as part of your matrix
to the baseline.  You may compare multiple models and multiple
clustering algorithms here.

In [25]:
class KmeansTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, k=4):
        self.km = KMeans(n_clusters=k,n_init=20)
        
    def transform(self, X, *_):
        labels = self.km.predict(X)
        enc = OneHotEncoder(categories='auto')
        oh_labels = enc.fit_transform(labels.reshape(-1,1))
        oh_labels = oh_labels.todense()
        return(np.hstack((X,oh_labels)))

    def fit(self,X,y=None,*_):
        self.km.fit(X)
        labels = self.km.predict(X)
        self.silhouette_score = round(silhouette_score(X,labels,metric='mahalanobis'),3)
        return(self)

class GmmTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, k=4):
        self.gmm = BayesianGaussianMixture(n_components=k,covariance_type='full',
                                           max_iter=500,n_init=10,warm_start=True)        
    def transform(self, X,*_):
        probs = self.gmm.predict_proba(X) + np.finfo(float).eps
        return(np.hstack((X,probs)))
        
    def fit(self,X,y=None,*_):
        self.gmm.fit(X)
        labels = self.gmm.predict(X)
        self.silhouette_score = round(silhouette_score(X,labels,metric='mahalanobis'),3)
        return(self)
    
## example for GMM  
preprocessor.fit(X_train)
X_train_pre = preprocessor.transform(X_train)    
gt = GmmTransformer(4)
gt.fit(X_train_pre)
X_train_gmm = gt.transform(X_train_pre)

print("Check how the feature space is increased by the cluster dummy obtained via the clustering algo.\n{}\n".format("-"*35))
print(X_train_pre.shape); print(X_train_gmm.shape)

## example for kmeans
preprocessor.fit(X_train)
X_train_pre = preprocessor.transform(X_train)    
kt = KmeansTransformer(4)
kt.fit(X_train_pre)
X_train_kmeans = kt.transform(X_train_pre)


print("\n{}\n{}".format(X_train_pre.shape, X_train_kmeans.shape))

Check how the feature space is increased by the cluster dummy obtained via the clustering algo.
-----------------------------------

(750, 7)
(750, 11)

(750, 7)
(750, 11)


In [27]:
def run_clustering_pipeline(smodel,umodel):
    fscores,sscores = [],[]
    for n_clusters in np.arange(3,8):
        
        if smodel == 'rf':
            clf = RandomForestClassifier(n_estimators=best_params['rf__n_estimators'],
                                         max_depth=best_params['rf__max_depth'],
                                         criterion=best_params['rf__criterion'])
        elif smodel == 'svm':
            clf = SVC(C=best_params["svm__C"],
                      gamma=best_params["svm__gamma"])
        else:
            raise Exception("invalid supervised learning model")
        
        if umodel == 'gmm':
            cluster = GmmTransformer(n_clusters)
            
        elif umodel == 'kmeans':
            cluster = KmeansTransformer(n_clusters)
            
        else:
            raise Exception("invalid unsupervised learning model")
        
        pipe = Pipeline(steps=[('pre', preprocessor),
                               ('clustering', cluster),
                               ('classifier', clf)])  
        
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        score = round(f1_score(y_test, y_pred, average='binary'),3)
        fscores.append(score)
        sscores.append(pipe['clustering'].silhouette_score)
        
    return(fscores)

## run the different iteration of the model
cp_results = {}
cp_results['svm-kmeans'] = run_clustering_pipeline('svm','kmeans')
cp_results['svm-gmm'] = run_clustering_pipeline('svm','gmm')
cp_results['rf-kmeans'] = run_clustering_pipeline('rf','kmeans')
cp_results['rf-gmm'] = run_clustering_pipeline('rf','gmm')

## display table of results
df_cp = pd.DataFrame(cp_results)
df_cp["n_clusters"] = [str(i) for i in np.arange(3,8)]
df_cp.set_index("n_clusters",inplace=True)
df_cp.head(n=10)


,svm-kmeans,svm-gmm,rf-kmeans,rf-gmm
n_clusters,,,,
3,0.538,0.538,0.538,0.534
4,0.538,0.538,0.533,0.542
5,0.538,0.538,0.525,0.538
6,0.538,0.538,0.524,0.538
7,0.538,0.538,0.516,0.520


In [33]:
## include it in a post about accuracy metrics.
print(classification_report(y_test, y_pred, target_names = ['subscriber', 'churn']))

              precision    recall  f1-score   support

  subscriber       0.80      0.92      0.86       178
       churn       0.68      0.44      0.54        72

    accuracy                           0.78       250
   macro avg       0.74      0.68      0.70       250
weighted avg       0.77      0.78      0.76       250



In [38]:
import collections

print (collections.Counter(y).items()) 

dict_items([(0.0, 711), (1.0, 289)])


### SOLUTION NOTE

`svm-kmeans` performs at baseline while `svm-gmm` performs below.  The
random forests model potentially sees a small improvement with the
addition of clusters.  This is a fairly small dataset with a small
number of features.  The utility of adding clustering to the pipeline
is generally more apparent in higher dimensional data sets.

## QUESTION 4

Run an experiment to see if you can you improve on your workflow with
the addition of re-sampling techniques?

In [ ]:
## YOUR CODE HERE

def run_clustering_pipeline(smodel,umodel):
    fscores,sscores = [],[]
    for n_clusters in np.arange(3,8):
        
        if smodel == 'rf':
            clf = RandomForestClassifier(n_estimators=best_params['rf__n_estimators'],
                                         max_depth=best_params['rf__max_depth'],
                                         criterion=best_params['rf__criterion'])
        elif smodel == 'svm':
            clf = SVC(C=best_params["svm__C"],gamma=best_params["svm__gamma"])
        else:
            raise Exception("invalid supervised learning model")
        
        if umodel == 'gmm':
            cluster = GmmTransformer(n_clusters)    
        elif umodel == 'kmeans':
            cluster = KmeansTransformer(n_clusters)
        else:
            raise Exception("invalid unsupervised learning model")
        
        pipe = pl.Pipeline(steps=[('pre', preprocessor),
                                  ('clustering', cluster),
                                  ('smote', SMOTE(random_state=42)),
                                  ('classifier', clf)])  
           
        pipe.fit(X_train,y_train) 
        y_pred = pipe.predict(X_test)    
        score = round(f1_score(y_test, y_pred,average='binary'),3)
        fscores.append(score)
        sscores.append(pipe['clustering'].silhouette_score)
        
    return(fscores)

## run the different iteration of the model
cp_results = {}
cp_results['svm-kmeans'] = run_clustering_pipeline('svm','kmeans')
cp_results['svm-gmm'] = run_clustering_pipeline('svm','gmm')
cp_results['rf-kmeans'] = run_clustering_pipeline('rf','kmeans')
cp_results['rf-gmm'] = run_clustering_pipeline('rf','gmm')

## display table of results
df_cp = pd.DataFrame(cp_results)
df_cp["n_clusters"] = [str(i) for i in np.arange(3,8)]
df_cp.set_index("n_clusters",inplace=True)
df_cp.head(n=10)

### SOLUTION NOTE

The inclusion of customer profiles does not significantly improve the overall model performance pipeline for either model.  There may be some minor improvement depending on the random seed, but since it does not degrade model performance either it can be useful in the context of marketing.  The clusters are customer profiles that are tied to predictive performance.  The re-sampling does help the random forest classifiers obtain similar performance results to SVM in this case.